## Pré processamento

In [1]:
import numpy as np
import pandas as pd

k = 2
df = pd.read_csv('salario.csv')

df.drop('id', axis=1, inplace=True)
df.drop('salario', axis=1, inplace=True)
df.insert(0, 'localizacao', '*')

for i in df.index:
    # Codificando gênero:
    if df.at[i, 'genero'] == 'Female':
        df.at[i, 'genero'] = 0
    elif df.at[i, 'genero'] == 'Male':
        df.at[i, 'genero'] = 1
        
    # Separando data:
    df.at[i, 'data'] = df.at[i, 'data'].split('/')
    
    # Unindo localização:
    c = df.at[i, 'cidade']
    e = df.at[i, 'estado']
    p = df.at[i, 'pais']
    
    df.at[i, 'localizacao'] = [c, e, p]
    
# Removendo colunas extras: cidade, estado e país
df.drop('cidade', axis=1, inplace=True)
df.drop('estado', axis=1, inplace=True)
df.drop('pais', axis=1, inplace=True)
        
df.head()
# len(df)
# print(df.iloc[0])
# print(df.at[130, 'localizacao'])
# aux = df.iloc[0]
# print(aux['localizacao'][0])

,localizacao,genero,data
0,"[Mobile, Alabama, United States]",1,"[10, 01, 1987]"
1,"[Rego, Porto, Portugal]",1,"[29, 11, 2007]"
2,"[Emmaboda, Kalmar, Sweden]",1,"[23, 06, 1984]"
3,"[Sala, Västmanland, Sweden]",0,"[04, 11, 1983]"
4,"[Quimperlé, Bretagne, France]",0,"[18, 11, 1990]"


## Distância entre registros:

In [2]:
def dist (r1, r2):
    dist = 0
    
    loc1 = df.at[r1, 'localizacao']
    loc2 = df.at[r2, 'localizacao']
    gen1 = df.at[r1, 'genero']
    gen2 = df.at[r2, 'genero']
    data1 = df.at[r1, 'data']
    data2 = df.at[r2, 'data']
    
    if gen1 != gen2: dist += 1
    
    if loc1[2] != loc2[2]: dist += 3
    elif loc1[1] != loc2[1]: dist += 2
    elif loc1[0] != loc2[0]: dist += 1
        
    if data1[2] != data2[2]: dist += 3
    elif data1[1] != data2[1]: dist += 2
    elif data1[0] != data2[0]: dist +=1
        
    return dist

## Verificando se dataset é anônimo:

In [3]:
def is_anonymous(ds, k):
    anonymity = True
    
    for i in ds.index:
        counter = 1
        for j in ds.index:
            if i == j: continue
            if dist(i, j) == 0: counter += 1
        if counter < k: anonymity = False
    
    return anonymity

## Clusterização dos registros:

In [4]:
def clustering (ds):
    clusters = []    # cluster_lista de clusters (cluster_lista de dicionários)
    checked = []     # lista de index dos registros já clusterizados
    
    for i in ds.index:
        if i not in checked:
            group = []    # cria novo cluster
            aux = {'index': i, 'dist': 0}
            checked.append(i)
            group.append(aux)
            
            for j in ds.index:
                if j not in checked:
                    d = dist(i, j)
                    if len(group) < k:    # se cluster não estiver cheio
                        aux = {'index': j, 'dist': d}
                        group.append(aux)
                    else:
                        max_dist = 0
                        max_id = 0
                        for item in group:    # procura item com maior distância no cluster
                            if item['dist'] > max_dist: 
                                max_dist = item['dist']
                                max_id = group.index(item)
                        if d < max_dist:    # substituindo item com maior distância por j
                            aux = {'index': j, 'dist': d}
                            group[max_id] = aux
                            
            for item in group: 
                if item not in checked: checked.append(item['index'])
            clusters.append(group)
            
    return clusters, checked
                    

# testando função:
# clusters = clustering(df)
# for cluster in clusters: print(cluster)

## Generalizando cluster_lista de k registros:

In [5]:
# 'cluster_list' é uma lista de dicionários {index, dist}

def generalize(cluster_list):
    checked = []
    diff_cidade = False
    diff_estado = False
    diff_pais = False
    diff_genero = False
    diff_dia = False
    diff_mes = False
    diff_ano = False
    
    # Percorrendo cluster e comparando registros 2 a 2:
    for item1 in cluster_list:
        i1 = cluster_list[cluster_list.index(item1)]['index']    # pegando valor 'index' do elemento item (dicionário)
        for item2 in cluster_list:
            i2 = cluster_list[cluster_list.index(item2)]['index']
            if i2 <= i1: continue
                
            r1 = df.iloc[i1]
            r2 = df.iloc[i2]
                
            # Verificando atributos com 2 valores diferentes dentro do cluster:
            if not diff_cidade:
                c1 = r1['localizacao'][0]
                c2 = r2['localizacao'][0]
                if c1 != c2: diff_cidade = True
            if not diff_estado:
                e1 = r1['localizacao'][1]
                e2 = r2['localizacao'][1]
                if e1 != e2: diff_estado = True
            if not diff_pais:
                p1 = r1['localizacao'][2]
                p2 = r2['localizacao'][2]
                if p1 != p2: diff_pais = True
            if not diff_genero:
                g1 = r1['genero']
                g2 = r2['genero']
                if g1 != g2: diff_genero = True
            if not diff_dia:
                d1 = r1['data'][0]
                d2 = r2['data'][0]
                if d1 != d2: diff_dia = True
            if not diff_mes:
                m1 = r1['data'][1]
                m2 = r2['data'][1]
                if m1 != m2: diff_mes = True
            if not diff_ano:
                a1 = r1['data'][2]
                a2 = r2['data'][2]
                if a1 != a2: diff_ano = True
                        
    # Suprimindo atributos com diferenças:
    for item in cluster_list:
        i = cluster_list[cluster_list.index(item)]['index']
        r = df.iloc[i]
        
        if diff_genero: r['genero'] = '*'
        if diff_pais:
            r['localizacao'][2] = '**'
            r['localizacao'][1] = '**'
            r['localizacao'][0] = '**'
        elif diff_estado:
            r['localizacao'][1] = '**'
            r['localizacao'][0] = '**'
        elif diff_cidade:
            r['localizacao'][0] = '**'   
        if diff_ano:
            r['data'][2] = '****'
            r['data'][1] = '**'
            r['data'][0] = '**'
        elif diff_mes:
            r['data'][1] = '**'
            r['data'][0] = '**'
        elif diff_dia:
            r['data'][0] = '**'
        


# Testando função:
# aux1 = {'index': 0, 'dist': 0}
# aux2 = {'index': 17, 'dist': 3}
# cluster_list = [aux1, aux2]
# for i in cluster_list:
#     ii = cluster_list[cluster_list.index(i)]['index']
#     r = df.iloc[ii]
#     print(r)
# generalize(cluster_list)
# for i in cluster_list:
#     ii = cluster_list[cluster_list.index(i)]['index']
#     r = df.iloc[ii]
#     print(r)

## Preparando novo dataset:

In [6]:
def get_new(old_df):
    new_df = pd.read_csv('salario.csv')
    new_df.drop('id', axis=1, inplace=True)
    new_df.drop('cidade', axis=1, inplace=True)
    new_df.drop('estado', axis=1, inplace=True)
    new_df.drop('pais', axis=1, inplace=True)
    new_df.insert(0, 'localizacao', '*')
    
#     new_df.head()
    
    for i in new_df.index:
        
    # formatando dados do dataset antigo:
        old_loc = old_df.at[i, 'localizacao']
        old_date = old_df.at[i, 'data']
        new_loc = old_loc[0] + ',' + old_loc[1] + ',' + old_loc[2]
        new_date = old_date[0] + '/' + old_date[1] + '/' + old_date[2]
        if old_df.at[i, 'genero'] == 0: new_gender = 'Feminino'
        elif old_df.at[i, 'genero'] == 1: new_gender = 'Masculino'
        elif old_df.at[i, 'genero'] == '*': new_gender = '*'
            
    # substituindo dados formatados no novo dataset:
        new_df.at[i, 'localizacao'] = new_loc
        new_df.at[i, 'genero'] = new_gender
        new_df.at[i, 'data'] = new_date
        
    return new_df

## Main:

In [7]:
clusters, checked = clustering(df)


# como a função de clusterização cria grupos de exatos k elementos, precisamos alocar possíveis registros não agrupados
for i in df.index:
    if not i in checked:   # se registro não clusterizado, vamos procurar elemento mais próximo dele
        min_dist = 0
        closest_neighb = i
        for j in df.index:
            dist = dist(i, j)
            if dist < min_dist:
                min_dist = dist
                closest_neighb = j
                
        for cluster in clusters:    # procurando cluster do elemento mais próximo
            for item in cluster:
                if item['index'] == closest_neighb:    
                    aux = {'index': i, 'dist': min_dist}
                    cluster.append(aux)
                    checked.append(i)
                    break
            break
            

for cluster in clusters:
    generalize(cluster)
            

new_df = get_new(df)
if is_anonymous(new_df, k): print('Dataset anonimizado com sucesso.')
export_csv = new_df.to_csv(r'salario_' + str(k) + '.csv', index=None, header=True)